In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dir_targets = '/kaggle/input/playerid-and-targets/results/targets_df_'
train_df = pd.read_csv('/kaggle/input/mlb-player-digital-engagement-forecasting/train.csv')
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)
train_df.tail()

In [ ]:
targets_1211 = unpack_json(train_df['nextDayPlayerEngagement'].iloc[1211])
playerBoxScores = unpack_json(train_df['playerBoxScores'].iloc[1211])

In [ ]:
targets_1211

In [ ]:
playerBoxScores

In [ ]:
playerBoxScores.info()

object형 전부 코드가 있으므로 버려도 될듯, 날짜는 우선 버리기, playerId를 기준으로 취합


object : gameDate, gameTimeUTC, teamName, playerName, positionName, positionType

In [ ]:
playerScore_target = pd.merge(targets_1211,playerBoxScores, how='outer',on='playerId')

In [ ]:
pd.set_option('max_rows', 10)

In [ ]:
playerScore_target

게임 날짜가 null인 것은 없으므로 'gameDate'가 NaN이면 그 행 삭제하기

In [ ]:
playerScore_target = playerScore_target.dropna(subset=['gameDate'])

object형 제거

In [ ]:
playerScore_target.drop(['gameDate', 'gameTimeUTC', 'teamName', 'playerName', 'positionName', 'positionType'], axis = 1, inplace = True)

In [ ]:
playerScore_target

In [ ]:
playerScore_target = playerScore_target.fillna(0)

In [ ]:
pd.set_option('max_columns', None)
playerScore_target

In [ ]:
playerScore_target.info()

In [ ]:
playerScore_target_corr = playerScore_target.corr()

In [ ]:
for i in ['groundIntoTriplePlay','pickoffs','pickoffsPitching']:
    print(playerScore_target[i].value_counts())

In [ ]:
playerScore_target_corr['target1'].sort_values(ascending = False)

rbi(득점타)가 target1에 어느정도 영향을 미치는 것 같음, 
1211 index에서 target1과 target2간의 연관도가 높게 측정됨 => 날짜에 따라서 각 target값들간의 연관도가 달라지는 것이 아닌가 하는 의심이 듬


target값과 관련된 NaN이 3개 있음 => 해결할 필요가 있어보임  => 모든 값이 0이어서 그럼 어쩔수 없다고 느껴짐

In [ ]:
'''
한개의 corr_matrix를 만들기

targets_1211 = unpack_json(train_df['nextDayPlayerEngagement'].iloc[1211])
playerBoxScores = unpack_json(train_df['playerBoxScores'].iloc[1211])
playerScore_target = pd.merge(targets_1211,playerBoxScores, how='outer',on='playerId')
playerScore_target = playerScore_target.dropna(subset=['gameDate'])
playerScore_target.drop(['gameDate', 'gameTimeUTC', 'teamName', 'playerName', 'positionName', 'positionType'], axis = 1, inplace = True)
playerScore_target = playerScore_target.fillna(0)
playerScore_target_corr = playerScore_target.corr()


'''


In [ ]:
# train_df속 playerBoxScores가 NaN인 값 걸러내기
train_df = train_df.dropna(subset=['playerBoxScores'])

In [ ]:
train_df.info()

In [ ]:
train_df=train_df.reset_index()


In [ ]:
train_df.drop(['index'], axis = 1, inplace = True)

In [ ]:
train_df

In [ ]:
# train_df 전처리 먼저하기
train_df = pd.read_csv('/kaggle/input/mlb-player-digital-engagement-forecasting/train.csv')
train_df = train_df.dropna(subset=['playerBoxScores'])
train_df=train_df.reset_index()
train_df.drop(['index'], axis = 1, inplace = True)



In [ ]:
train_df

In [ ]:

def playerBoxScores_targets_corr(train_df, start, last):
    plus = lambda a, b : a + b
    for i in range(start, last):
        targets = unpack_json(train_df['nextDayPlayerEngagement'].iloc[i])
        playerBoxScores = unpack_json(train_df['playerBoxScores'].iloc[i])
        playerScore_targets = pd.merge(targets, playerBoxScores, how='outer',on='playerId')
        playerScore_targets = playerScore_targets.dropna(subset=['gameDate'])
        playerScore_targets.drop(['gameDate', 'gameTimeUTC', 'teamName', 'playerName', 'positionName', 'positionType'], axis = 1, inplace = True)
        playerScore_targets = playerScore_targets.fillna(0)
        playerScore_target_corr = playerScore_targets.corr()
        if i == start:
            p_t_c = playerScore_target_corr
        else:
            p_t_c = p_t_c.combine(playerScore_target_corr, plus)
        if i%50 == 0:
            print(i)
    return p_t_c
 

In [ ]:
playerScore_target_corr = playerBoxScores_targets_corr(train_df, 0, 538)

In [ ]:
playerScore_target_corr = playerScore_target_corr.apply(lambda x : x/538)

In [ ]:
playerScore_target_corr_1 = playerScore_target_corr.loc['target1'].sort_values(ascending = False)

In [ ]:
playerScore_target_corr_1[:10]

In [ ]:
playerScore_target_corr_2 = playerScore_target_corr.loc['target2'].sort_values(ascending = False)
playerScore_target_corr_2[:10]

In [ ]:
playerScore_target_corr_3 = playerScore_target_corr.loc['target3'].sort_values(ascending = False)
playerScore_target_corr_3[:10]

In [ ]:
playerScore_target_corr_4 = playerScore_target_corr.loc['target4'].sort_values(ascending = False)
playerScore_target_corr_4[:10]

playerBoxScores가 NaN인 날을 제외하고 corr을 했더니 target간의 연관성이 높아짐

타겟을 제외한 경우 0.2정도의 연관성을 보이는 것들이 몇개 있음 => 그러한 특징들이 게임 관련이므로 선수 포지션과 연관지어 다시한번 연관성을 책정하면 유의미한 결과가 나올것으로 예상